<a href="https://colab.research.google.com/github/tonykipkemboi/convergence-2025-demo/blob/main/Convergence_2025_Talk_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Crafting Effective Agents - from Design to Observability**

---
## Convergence 2025
## Tony Kipkemboi - Sr. Developer Advocate @CrewAI
[@tonykipkemboi](https://www.linkedin.com/in/tonykipkemboi)

## Download Dependencies

In [1]:
!pip install --q crewai crewai-tools opik

## Configure API Keys

----
- [Gemini API Key](https://aistudio.google.com/app/apikey)
- [OpenAI API Key](https://platform.openai.com/)
- [Comet/Opik API Key](https://www.comet.com/signup)


In [2]:
from google.colab import userdata
import os

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
COMET_API_KEY = userdata.get('COMET_API_KEY')
COMET_WORKSPACE = userdata.get('COMET_WORKSPACE')
SERPER_API_KEY = userdata.get('SERPER_API_KEY')

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY
os.environ["COMET_API_KEY"] = COMET_API_KEY
os.environ["COMET_WORKSPACE"] = COMET_WORKSPACE
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

## Build the Agents (Agents + Tasks + Tools = Crew)


---
**Demo Purpose**

*To demonstrate how specialized AI agents with distinct roles can collaborate to produce sophisticated financial analysis using real market data, while showcasing end-to-end observability of multi-agent systems.*

In [3]:
import os
import time
import logging
from datetime import datetime

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Import CrewAI, Opik, and other tools
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
import opik
from opik.integrations.crewai import track_crewai

# Import IPython display components
from IPython.display import Markdown, HTML, display
import ipywidgets as widgets


# Configure Opik & LLM
opik.configure(
    api_key=COMET_API_KEY,
    use_local=False,
    workspace=COMET_WORKSPACE
  )
llm = LLM(
    model="gemini/gemini-2.5-flash-preview-04-17"
)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:623: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.11/dist-packages/opik/error_tracking/shutdown_hooks.py:12: SentryHubDeprecationWarning: `sentry_sdk.Hub` is deprecated and will be removed in a future major release. Please consult our 1.x to 2.x migration guide for details on how to migrate `Hub` usage to the new API: https://docs.sentry.io/platforms/python/migration/1.x-to-2.x
  client = sentry_sdk.Hub.current.client
OPIK: Opik is already configured. You can check the settings by viewing the config file at /root/.opik.config


### Agents and Tasks

In [4]:
def create_investment_analysis_system(company_name, ticker, year):
    """
    Create the investment analysis system using CrewAI components with web tools.

    Args:
        company_name: Name of the company to analyze
        ticker: Stock ticker symbol
        year: The year to analyze (for context)
    """
    logger.info(f"Creating investment analysis system for {company_name} ({ticker}) for year {year}")

    # Initialize the tools
    serper_tool = SerperDevTool()
    scrape_tool = ScrapeWebsiteTool()

    # Create specialized agents following the Role-Goal-Backstory framework
    financial_analyst = Agent(
        role="Financial Data Analyst specializing in SaaS metrics",
        goal=f"Extract and analyze key financial data points for {year} with a focus on accuracy and context",
        backstory=f"""With 12 years of experience in financial analysis at top investment firms,
        you've developed a methodical approach to financial data extraction and normalization.
        You focus on SaaS-specific metrics and have a talent for identifying early warning signs
        in financial statements. Your analysis always includes both GAAP and non-GAAP metrics
        with a particular emphasis on cash flow and unit economics. You're analyzing data for the year {year}.""",
        verbose=True,
        allow_delegation=False,
        tools=[serper_tool, scrape_tool],
        llm=llm
    )

    industry_expert = Agent(
        role="SaaS Industry Specialist with deep competitive knowledge",
        goal=f"Provide industry context and competitive positioning insights for {year}",
        backstory=f"""Having covered the SaaS industry for 15 years as both an analyst and
        consultant, you understand the nuances of different business models and competitive
        dynamics. You've worked with venture capital firms evaluating hundreds of SaaS
        companies and can quickly identify the key differentiators and challenges in any
        SaaS market segment. You maintain a database of competitive intelligence on all
        major SaaS players. You're analyzing the industry landscape for {year}.""",
        verbose=True,
        allow_delegation=False,
        tools=[serper_tool, scrape_tool],
        llm=llm
    )

    valuation_expert = Agent(
        role="SaaS Valuation Analyst with DCF modeling expertise",
        goal=f"Determine fair value using multiple methodologies and sensitivity analysis for {year}",
        backstory=f"""After working at a top investment bank for 10 years in the software
        equity research division, you've developed sophisticated models for valuing high-growth
        SaaS companies. You recognize that traditional P/E metrics don't work well for many
        SaaS businesses and instead use a combination of DCF analysis, Rule of 40 framework,
        and peer-based multiples. You're especially skilled at creating scenario-based models
        that account for the uncertainty in projections. You're performing a valuation based on {year} data.""",
        verbose=True,
        allow_delegation=False,
        tools=[serper_tool, scrape_tool],
        llm=llm
    )

    risk_analyst = Agent(
        role="Investment Risk Specialist focused on technology companies",
        goal=f"Identify and quantify potential risks and their probability for {year}",
        backstory=f"""With a background in both technology and risk management, you've
        developed systems for quantifying risks in high-growth technology investments.
        You previously led risk assessment for a technology-focused hedge fund where
        you created a framework for assessing market, financial, operational, technological,
        and regulatory risks. You use both qualitative analysis and quantitative
        Monte Carlo simulations to model potential outcomes and risk factors. You're assessing risks based on {year} data.""",
        verbose=True,
        allow_delegation=False,
        tools=[serper_tool, scrape_tool],
        llm=llm
    )

    report_writer = Agent(
        role="Investment Report Writer specializing in technology stocks",
        goal=f"Create comprehensive, clear investment recommendations with supporting evidence for {year}",
        backstory=f"""After 8 years writing investment research reports at major financial
        institutions, you've mastered the art of distilling complex analysis into clear,
        compelling investment narratives. You're known for creating balanced analysis that
        presents both bull and bear cases, with a focus on thesis, valuation, catalysts,
        and risks. Your reports always include clear action items and price targets based
        on probability-weighted scenarios. You're writing a report based on {year} data and analysis.""",
        verbose=True,
        allow_delegation=False,
        llm=llm
    )

    # Create tasks with detailed descriptions and expected outputs
    # Following the 80/20 rule - focusing on detailed task design
    financial_analysis_task = Task(
        description=f"""
        Analyze {company_name}'s ({ticker}) financial performance for {year} focusing on:
        1. Revenue growth (YoY and 5-year CAGR)
        2. Profit margin trends (last 8 quarters)
        3. Cash flow stability (operating and free cash flow)
        4. SaaS-specific metrics (ARR, CAC, LTV, churn)
        5. Balance sheet health and liquidity

        Use the SerperDevTool to search for real financial data for {ticker}.
        Search for specific queries like "{ticker} financial statements {year}",
        "{ticker} annual report {year}", and "{ticker} investor relations earnings".

        Use the ScrapeWebsiteTool to extract detailed data from financial websites.
        Try scraping from sites like:
        - The company's investor relations page (search for "{company_name} investor relations")
        - SEC EDGAR database (use css selectors to extract financial tables)
        - Financial news sites with analysis of {company_name}

        Your analysis should incorporate real data from these tools, comparing key metrics
        to SaaS industry benchmarks where applicable. Use a first-principles approach to
        evaluate the financial health of the business.

        Focus on data and performance specific to the year {year}.

        Important: Format your analysis in markdown with clear sections and bullet points.
        """,
        expected_output=f"""Financial Analysis for {company_name} ({ticker}) for {year}""",
        agent=financial_analyst
    )

    industry_analysis_task = Task(
        description=f"""
        Analyze {company_name}'s competitive positioning in the SaaS industry for {year}:
        1. Market share and position in its primary markets
        2. Competitive advantages and moats that differentiate it
        3. Comparison to key competitors (at least 3 direct competitors)
        4. Industry trends and their specific impact on the company
        5. Potential disruptive threats to its market position

        Use the SerperDevTool to search for industry analyses, market reports,
        and competitor information. Search for queries like:
        - "{company_name} market share {year}"
        - "{company_name} competitors analysis"
        - "SaaS industry trends {year}"
        - "{company_name} competitive advantage"

        Use the ScrapeWebsiteTool to extract detailed content from:
        - Industry analyst reports (Gartner, Forrester, IDC)
        - Competitor websites and investor presentations
        - Tech blogs and industry publications

        Use the financial metrics from the financial analysis as context when
        evaluating competitive positioning and market dynamics.

        Focus on the competitive landscape and industry dynamics specific to {year}.

        Important: Format your analysis in markdown with clear sections and bullet points.
        """,
        expected_output=f"""Competitive Analysis for {company_name} in the SaaS industry for {year}""",
        agent=industry_expert
    )

    valuation_task = Task(
        description=f"""
        Determine {company_name}'s fair value using multiple methodologies for {year}:
        1. Discounted Cash Flow (DCF) analysis with detailed assumptions
        2. Relative valuation using appropriate multiples (EV/Revenue, Rule of 40, etc.)
        3. Peer group comparison with at least 4 comparable companies
        4. Sum-of-parts analysis if applicable to this business

        Use the SerperDevTool to search for:
        - "{ticker} valuation models {year}"
        - "{ticker} analyst price targets {year}"
        - "{company_name} comparable companies"
        - "SaaS valuation multiples {year}"

        Use the ScrapeWebsiteTool to extract data from:
        - Equity research reports (if available)
        - Financial websites with valuation data
        - Investor presentations discussing growth projections

        Use the financial analysis and industry analysis as context for your valuation.
        Be explicit about your assumptions for:
        - Growth rates (short-term and terminal)
        - Margin expansion trajectory
        - Discount rates (WACC)
        - Multiple compression/expansion

        Ensure your valuation is based on data and market conditions for {year}.

        Important: Format your analysis in markdown with clear sections and bullet points.
        Include a clear fair value range and whether the current stock price is under/overvalued.
        """,
        expected_output=f"""Fair Value Analysis for {company_name} ({ticker}) for {year}""",
        agent=valuation_expert
    )

    risk_assessment_task = Task(
        description=f"""
        Conduct a comprehensive risk assessment for {company_name} for {year}:
        1. Market risks (competition, market changes, TAM limitations)
        2. Financial risks (profitability path, SBC dilution, cash burn)
        3. Operational risks (customer concentration, geographical exposure)
        4. Technological risks (technical debt, disruption, obsolescence)
        5. Regulatory risks (data privacy, antitrust, sector-specific)

        Use the SerperDevTool to search for:
        - "{ticker} risk factors"
        - "{company_name} challenges {year}"
        - "SaaS industry risks {year}"
        - "Technology regulation impact on {company_name}"

        Use the ScrapeWebsiteTool to extract detailed content from:
        - SEC filings (10-K, 10-Q) which contain risk disclosures
        - Industry reports discussing sector risks
        - News articles about challenges facing {company_name}

        For each risk type:
        - Assign a risk level (LOW, MEDIUM, HIGH)
        - Explain the rationale with specific evidence
        - Discuss potential mitigating factors
        - Where possible, quantify the potential impact

        Use all previous analyses as context for your risk assessment.
        Focus on risks relevant to {year} and near-term future.

        Important: Format your analysis in markdown with clear sections and bullet points.
        Conclude with an overall risk-reward assessment.
        """,
        expected_output=f"""Risk Assessment for {company_name} for {year}""",
        agent=risk_analyst
    )

    investment_report_task = Task(
        description=f"""
        Create a comprehensive investment recommendation for {company_name} for {year}:
        1. Executive summary with clear rating (BUY, HOLD, SELL)
        2. Investment thesis with 3-5 key points supporting your recommendation
        3. Financial projection summary for the next 3 years
        4. Valuation methodology and price target with upside/downside percentage
        5. Risk assessment summary with key watch points
        6. Detailed recommendation with time horizon and position sizing guidance

        Use the ScrapeWebsiteTool to search for any final pieces of information:
        - "Latest news on {company_name}"
        - "{ticker} analyst recommendations {year}"
        - "{company_name} future outlook"

        Use all previous analyses as context and ensure all claims are supported by data.
        Your recommendation should be balanced, considering both bull and bear cases.
        Make sure your investment recommendation is appropriate for {year} market conditions.

        Important: Format your report in markdown with clear sections, bullet points,
        and a professional structure. This report will be used by investment professionals
        to make decisions, so clarity and precision are essential.

        Very important: DO NOT INCLUDE THE '```markdown' in the markdown final report so I can render it cleanly.
        """,
        expected_output=f"""Investment Report for {company_name} for {year}""",
        agent=report_writer
    )

    # Create the crew
    investment_crew = Crew(
        agents=[financial_analyst, industry_expert, valuation_expert, risk_analyst, report_writer],
        tasks=[financial_analysis_task, industry_analysis_task, valuation_task, risk_assessment_task, investment_report_task],
        process=Process.sequential,
        verbose=True
    )

    return investment_crew

## Run the Multi-agents

In [5]:
def run_investment_analysis(company_name, ticker, year):
    """
    Run the investment analysis with Opik integration.
    This is the main entry point for the demo.
    """
    logger.info(f"Starting investment analysis for {company_name} ({ticker}), {year}")

    # Enable Opik tracking for CrewAI
    track_crewai(project_name=f"investment-analysis-{company_name}")

    try:
        # Create the investment crew
        logger.info("Creating investment analysis system...")
        crew = create_investment_analysis_system(company_name, ticker, year)

        # Run the analysis
        logger.info(f"Executing investment analysis for {company_name} ({ticker})...")
        result = crew.kickoff(inputs={"company_name": company_name, "ticker": ticker, "year": year})

        # Save the result to a file in Colab
        result_text = result.raw if hasattr(result, 'raw') else str(result)
        output_file = f"{company_name}_{ticker}_{year}_analysis.md"
        with open(output_file, "w") as f:
            f.write(result_text)

        logger.info(f"Analysis completed and saved to {output_file}")

        return result, output_file

    except Exception as e:
        # Log the error
        logger.error(f"Error running analysis: {str(e)}")
        raise e


def run_demo_ui():
    company_input = widgets.Text(
        value='Snowflake',
        placeholder='Company Name',
        description='Company:',
        disabled=False
    )

    ticker_input = widgets.Text(
        value='SNOW',
        placeholder='Ticker Symbol',
        description='Ticker:',
        disabled=False
    )

    # Add year input with current year as default
    current_year = datetime.now().year
    year_input = widgets.IntText(
        value=current_year,
        placeholder='Year to analyze',
        description='Year:',
        disabled=False,
        min=2000,
        max=current_year
    )

    submit_button = widgets.Button(
        description='Run Analysis',
        button_style='success',
        tooltip='Click to start the analysis'
    )

    output = widgets.Output()

    display(company_input, ticker_input, year_input, submit_button, output)

    def on_button_clicked(b):
        with output:
            output.clear_output()
            print(f"Running investment analysis for {company_input.value} ({ticker_input.value}) for {year_input.value}...")
            print("This will take a few minutes to complete as it runs real LLM queries.")
            print("Progress is being logged to Opik dashboard.")

            try:
                result, output_file = run_investment_analysis(company_input.value, ticker_input.value, year_input.value)

                print("\n===== Analysis Complete =====\n")
                print(f"Full analysis saved to: {output_file}")

                # Display link to Opik dashboard
                display(HTML(f'<a href="https://www.comet.com/workspace" target="_blank">View detailed metrics in Opik Dashboard</a>'))

                # Display the result preview
                result_text = result.raw if hasattr(result, 'raw') else str(result)
                preview = result_text[:2000] + "..." if len(result_text) > 2000 else result_text
                display(Markdown("## Analysis Preview\n" + preview))

                # Option to display full result
                view_button = widgets.Button(
                    description='Show Full Analysis',
                    button_style='info',
                    tooltip='Click to view the complete analysis'
                )

                def on_view_clicked(b):
                    with output:
                        display(Markdown("## Complete Investment Analysis\n" + result_text))

                view_button.on_click(on_view_clicked)
                display(view_button)

            except Exception as e:
                print(f"\nError running analysis: {str(e)}")

    submit_button.on_click(on_button_clicked)

# Run the demo UI when this cell is executed
run_demo_ui()

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget.py:477: DeprecationWarning: Passing unrecognized arguments to super(IntText).__init__(placeholder='Year to analyze', min=2000, max=2025).
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


Text(value='Snowflake', description='Company:', placeholder='Company Name')

Text(value='SNOW', description='Ticker:', placeholder='Ticker Symbol')

IntText(value=2025, description='Year:')

Button(button_style='success', description='Run Analysis', style=ButtonStyle(), tooltip='Click to start the an…

Output()